In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#default_exp classify_image

In [3]:
#export
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
from food.psql import *

# !nbdev_build_lib
import requests
from food.clipmodel import image2clip,text2clip
from PIL import Image
import torch

In [4]:
#export
def search_by_clip(clip, topk=5, query_expr='',search_params = {}): #32768
    search_params = {"metric_type": "IP",
                     "params": search_params}

    print(query_expr)
    results = collection.search(clip[None,], "clip", search_params, limit=topk, expr=query_expr, output_fields=["id"])
    results = results[0]
    return list(results.distances), results.ids

def get_metadata(ids):
    q = f"""select * from foods where id in {tuple(ids)}"""
    return pd.read_sql(q,engine)

In [5]:
#export
def get_calories(url=None,path=None):
    r = search_image(url=url)
    r = r[(r['energy_kcal_100g']>r['energy_kcal_100g'].quantile(0.3))&(r['energy_kcal_100g']<r['energy_kcal_100g'].quantile(0.9))]
    rr = r[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']].mean().round(2)
    rr['ids'] =  r['id'].tolist()
    rr['scores'] =  r['score'].tolist()
    rr['names']  =  r['product_name'].tolist()
    return rr

In [7]:
#export
cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
foods = pd.read_sql('select * from foods',engine)
clip = torch.Tensor(foods['clip']).cuda(2).type(torch.float16)

In [11]:
#export
def search_image(url=None,path=None,milvus=False):
    if url:
        response = requests.get(url, stream=True)
        image = Image.open(response.raw)
    elif path:
        image = Image.open(path)
    image_clip = image2clip(image)
    if milvus:
        results = search_by_clip(clip.numpy())
        df = get_metadata(results[1])
        df['score'] = results[0]
    else:
        df = foods.drop(columns = ['clip'])
        df['score'] = cos(clip, torch.Tensor(image_clip).cuda(2)).cpu().detach().numpy()
        df = df.sort_values('score',ascending=False).head(1)

    return df

In [12]:
search_image('https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/classic-cheese-pizza-1617996277.jpg?crop=1.00xw:1.00xh;0,0&resize=980:*')

,id,description,category,energy,protein,carb,fat,score
4486,1101973,"Pizza, cheese, from restaurant or fast food, t...",Pizza,271.0,10.81,33.17,10.54,0.240645


In [14]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_recreate_milvus.ipynb.
Converted 00_tools.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted bot.ipynb.
Converted classfy_image.ipynb.
Converted image_segmentation.ipynb.
Converted inference.ipynb.
Converted milvus_search_test.ipynb.
Converted milvus_update.ipynb.
Converted openfoodfacts_json_parsing.ipynb.
Converted usda_parsing.ipynb.
